# 3D Skeleton Visualization System: Sprint 10 Report

## Overview

This report details the implementation of a 3D skeleton visualization system for comparing ground truth and predicted skeletal poses. 

## System Architecture

The system is composed of several React components working together:

1. **CSVPrediction** - The main container component that handles file uploading, data processing, and state management
2. **PoseNet3DVisualization** - Orchestrates the visualization components and manages display modes
3. **SkeletonRenderer** - Core Three.js component that renders 3D skeletons with rotation capabilities
4. **SkeletonContext & Provider** - Context API implementation for sharing state across components
5. **SkeletonControls** - UI controls for animation playback and view options
6. **AnimationManager** - Handles animation timing and frame synchronization

## Detailed Component Breakdown

### 1. CSVPrediction Component

This is the main entry point component that handles:

- **File Management**: CSV uploads via drag-and-drop or file selection
- **Backend Integration**: Loading sample ground truth data from the server
- **Data Processing**: Parsing CSV files with automatic delimiter detection
- **State Management**: Tracking data state, UI state, and visualization modes
- **UI Rendering**: Tab-based interface for data and visualization views

#### Key Features:

- Adaptive delimiter detection (comma or tab-separated values)
- Progress indication during file processing
- Data preview with Z-value highlighting
- Tab-based interface switching between data view and 3D visualization
- Toggle for switching between side-by-side and overlapping visualization modes

```javascript
// Example of CSV data processing function
const processCSV = async (file: File) => {
  // Read file content
  const text = await file.text();
  const lines = text.split("\n");
  
  // Detect delimiter (comma or tab)
  const firstLine = lines[0];
  const delimiter = firstLine.includes("\t") ? "\t" : ",";
  
  // Parse headers and rows...
  // Convert to format for 3D visualization...
  
  // Set as predicted data
  setPredictedData(objectRows);
};
```

### 2. PoseNet3DVisualization Component

This component orchestrates the visualization experience:

- Wraps child components in the SkeletonProvider context
- Determines visualization mode (side-by-side or overlapping)
- Manages synchronized rendering of multiple SkeletonRenderer instances
- Provides feedback for invalid skeletal data

```javascript
// Rendering logic for different modes
{showSideBySide ? (
  // Side-by-side view
  <div className="grid grid-cols-2 gap-4">
    <SkeletonRenderer
      poseData={poseData}
      isGroundTruth={true}
      label={groundTruthLabel}
    />
    <SkeletonRenderer
      poseData={predictedData}
      isGroundTruth={false}
      label={predictedLabel}
    />
  </div>
) : (
  // Overlapping view
  <SkeletonRenderer
    poseData={poseData}
    isGroundTruth={true}
    label={`${groundTruthLabel} vs ${predictedLabel}`}
    comparisonPoseData={predictedData}
  />
)}
```

### 3. SkeletonRenderer Component

This is the core visualization component built with Three.js:

- **3D Scene Setup**: Canvas creation, camera positioning, lighting
- **Skeleton Rendering**: Converting joint data into 3D geometry
- **Animation Support**: Handling frame updates with position preservation
- **Interaction**: Camera controls for panning, rotation, and zooming
- **Visual Feedback**: Color coding and labels for different data types

The component follows this initialization sequence:
1. Create THREE.js scene, camera, renderer, and lights
2. Set up OrbitControls for user interaction
3. Add visual elements like labels and legends
4. Create the animation loop
5. Set up event listeners and cleanups

```javascript
// Core part of the rendering logic
const updateSkeleton = (frameIndex) => {
  // Get frame data and create skeleton group
  const frameData = poseData[validFrameIndex];
  const skeletonGroup = new THREE.Group();
  
  // Extract joints from frame data
  for (const key in frameData) {
    if (key.endsWith("_x") || key.endsWith("_y") || key.endsWith("_z")) {
      // Extract joint coordinates...
    }
  }
  
  // Create joint spheres
  Object.keys(joints).forEach((jointName) => {
    // Create sphere for each joint...
    skeletonGroup.add(sphere);
  });
  
  // Create bone connections
  POSE_CONNECTIONS.forEach((connection) => {
    // Create cylinder between connected joints...
    skeletonGroup.add(bone);
  });
  
  // Add skeleton to scene
  sceneRef.current.add(skeletonGroup);
}
```

#### Key Technical Features:

1. **Rotation Management**:
   - Manual rotation via OrbitControls
   - Auto-rotation toggled by the user
   - Synchronized rotation between primary and comparison skeletons
   - Rotation persistence across frame changes

2. **Skeleton Visualization**:
   - Color-coded joints and bones (blue for ground truth, red for prediction)
   - Thicker bones and joints for better visibility
   - Synchronized positioning between comparison skeletons
   - Proper scaling and Y-flipping based on data characteristics

3. **Visual Aids**:
   - In-scene legend indicating ground truth vs prediction
   - Dynamic labeling based on view type
   - Label coloring to match the corresponding skeleton color

### 4. SkeletonContext and Provider

Implements React Context API for managing shared state:

```javascript
const SkeletonContext = createContext({
  currentFrame: 0,
  setCurrentFrame: (frame: number) => {},
  isPlaying: false,
  setIsPlaying: (isPlaying: boolean) => {},
  playbackSpeed: 1,
  setPlaybackSpeed: (speed: number) => {},
  autoRotate: false,
  setAutoRotate: (autoRotate: boolean) => {},
  hasSkeletonData: false,
  setHasSkeletonData: (hasData: boolean) => {},
});
```

Key shared state includes:
- Current animation frame
- Playback state (playing/paused)
- Playback speed
- Auto-rotation toggle
- Skeleton data availability flag

### 5. SkeletonControls Component

Provides user interface for controlling the visualization:

- **Playback Controls**: Play, pause, and reset buttons
- **Frame Navigation**: Seek bar and frame counter
- **Speed Controls**: Playback speed adjustment
- **View Options**: Auto-rotation toggle

```javascript
<div className="flex items-center justify-between">
  <div className="flex items-center space-x-2">
    <Button onClick={togglePlayback} variant="outline" size="sm">
      {isPlaying ? <Pause className="h-4 w-4" /> : <Play className="h-4 w-4" />}
    </Button>
    <Button onClick={resetAnimation} variant="outline" size="sm">
      <RotateCcw className="h-4 w-4" />
    </Button>
  </div>
  
  <Slider
    value={[currentFrame]}
    max={totalFrames - 1}
    step={1}
    onValueChange={handleSliderChange}
  />
  
  <div className="flex items-center space-x-2">
    <Label htmlFor="auto-rotate" className="text-sm">
      Auto Rotate
    </Label>
    <Switch
      id="auto-rotate"
      checked={autoRotate}
      onCheckedChange={setAutoRotate}
    />
  </div>
</div>
```

### 6. AnimationManager Component

Handles animation timing and synchronization:

- **requestAnimationFrame** loop management
- Frame rate control and timing
- Synchronization between multiple skeleton views
- Animation state persistence

```javascript
useEffect(() => {
  if (!isPlaying) return;
  
  let lastFrameTime = 0;
  const frameInterval = 1000 / (fps * playbackSpeed);
  
  const animate = (timestamp) => {
    if (timestamp - lastFrameTime >= frameInterval) {
      setCurrentFrame((prev) => {
        if (prev >= totalFrames - 1) {
          return 0; // Loop back to start
        }
        return prev + 1;
      });
      lastFrameTime = timestamp;
    }
    
    animationRef.current = requestAnimationFrame(animate);
  };
  
  animationRef.current = requestAnimationFrame(animate);
  
  return () => {
    if (animationRef.current) {
      cancelAnimationFrame(animationRef.current);
    }
  };
}, [isPlaying, playbackSpeed, totalFrames]);
```

## Technical Implementation Details

### CSV Data Processing

The system processes CSV data with the following approach:

1. **Reading & Parsing**: 
   - Detects and adapts to different delimiters (comma or tab)
   - Handles header detection and column mapping
   - Converts numeric values automatically

2. **Data Structure**:
   - Each frame becomes an object with properties for each joint coordinate
   - Properties follow the naming convention: `[joint_name]_[x|y|z]`
   - Z-values are optional and handled appropriately when present

3. **Validation**:
   - Checks for required coordinates and proper formatting
   - Validates joint structure consistency across frames
   - Provides feedback for missing or invalid data

### Three.js Implementation

The 3D rendering utilizes Three.js with these key techniques:

1. **Scene Setup**:
   - Dark-themed background with appropriate lighting
   - Perspective camera with controlled viewing parameters
   - OrbitControls for intuitive user interaction

2. **Skeleton Construction**:
   - Joints represented as spheres with appropriate colors
   - Bones implemented as oriented cylinders connecting joints
   - Dynamic scaling based on data characteristics
   - Y-flipping when coordinate systems differ

3. **Animation & Interaction**:
   - Smooth transitions between animation frames
   - Position and rotation preservation during updates
   - Synchronized rotation between compared skeletons
   - Responsive resizing with window dimensions

### UI/UX Design

The user interface is designed for clarity and ease of use:

1. **Tab-Based Navigation**:
   - Data view for uploading and previewing CSV files
   - 3D view for visualization and interaction
   - Seamless transition between views

2. **Data Upload Experience**:
   - Drag-and-drop support with visual feedback
   - File selection alternative via dialog
   - Sample data loading option for quick testing

3. **Visualization Controls**:
   - Toggle between side-by-side and overlapping views
   - Playback controls with speed adjustment
   - Frame-by-frame navigation via slider
   - Auto-rotation for better spatial perception

4. **Visual Feedback**:
   - Color-coded skeletons (blue for ground truth, red for prediction)
   - In-scene legend explaining the color scheme
   - Labels for clear identification
   - Error alerts for invalid data

## Example Usage Flow

1. User loads the application and sees the data tab
2. They either upload a CSV file or load sample data
3. The system processes the file and displays a preview
4. User switches to the 3D view tab to see the visualization
5. They can toggle between side-by-side and overlapping views
6. Animation controls allow them to play through the frames
7. Auto-rotation can be enabled for better spatial understanding
8. Frame by frame examination is possible via the slider


## Conclusion

The 3D Skeleton Visualization System provides a solution for comparing ground truth and predicted skeletal poses. By leveraging Three.js for rendering and React for UI/state management.